# nprobust: Python vs R Comparison

This notebook tests the Python implementation of the nprobust package and compares results with R.

**Instructions:**
1. Run this notebook to generate Python results
2. Run the companion R script `comparison_test.R` to generate R results
3. Compare the outputs side by side

In [1]:
# Import required packages
import numpy as np
import pandas as pd
import sys
sys.path.insert(0, '.')

from nprobust import lprobust, lpbwselect, kdrobust, kdbwselect

print("nprobust package loaded successfully!")

nprobust package loaded successfully!


## Generate Test Data

We use the same random seed as R to ensure identical data.

In [7]:
# Set seed for reproducibility (same as R)
np.random.seed(42)

# Generate data
df = pd.read_csv('test_data_r.csv')
x = df.x.values
y = df.y.values

print(f"Generated {n} observations")
print(f"x range: [{x.min():.4f}, {x.max():.4f}]")
print(f"y range: [{y.min():.4f}, {y.max():.4f}]")
print(f"\nFirst 5 observations:")
print(pd.DataFrame({'x': x[:5], 'y': y[:5]}))

Generated 500 observations
x range: [0.0002, 0.9966]
y range: [-2.0313, 2.4828]

First 5 observations:
          x         y
0  0.914806 -1.058168
1  0.937075 -0.360622
2  0.286140  0.375082
3  0.830448 -0.779939
4  0.641746 -0.128605


---
## Test 1: lprobust with Fixed Bandwidth

Local polynomial regression with h=0.15, p=1, Epanechnikov kernel

In [9]:
# Define evaluation points (same as R)
eval_points = np.array([0.1, 0.25, 0.5, 0.75, 0.9])

# Run lprobust
result_lp = lprobust(y, x, eval=eval_points, h=0.15, p=1, kernel="epa", vce="nn")

print("="*70)
print("TEST 1: lprobust(y, x, eval=c(0.1,0.25,0.5,0.75,0.9), h=0.15, p=1)")
print("="*70)
result_lp.summary()

TEST 1: lprobust(y, x, eval=c(0.1,0.25,0.5,0.75,0.9), h=0.15, p=1)
Call: lprobust

Sample size (n)                              =    500
Polynomial order for point estimation (p)    =    1
Order of derivative estimated (deriv)        =    0
Polynomial order for confidence interval (q) =    2
Kernel function                              =    Epanechnikov
Bandwidth method                             =    Manual

                                     Point      Std.       Robust B.C.       
          eval         h   Eff.n      Est.     Error      [ 95% C.I. ]       
   1     0.100     0.150     132     0.547     0.041[  0.463 ,   0.677]
   2     0.250     0.150     154     0.889     0.051[  0.760 ,   1.103]
   3     0.500     0.150     152    -0.062     0.043[ -0.210 ,   0.040]
   4     0.750     0.150     130    -0.967     0.039[ -1.137 ,  -0.918]
   5     0.900     0.150     122    -0.572     0.054[ -0.768 ,  -0.457]
----------------------------------------------------------------------

In [ ]:
# Display raw estimates for easy comparison
print("\nRaw Estimates (for comparison with R):")
columns = ['eval', 'h', 'b', 'N', 'tau.us', 'tau.bc', 'se.us', 'se.rb']
df_lp = pd.DataFrame(result_lp.Estimate, columns=columns)
print(df_lp.to_string(index=False))

---
## Test 2: lprobust with Different Kernel (Uniform)

In [ ]:
result_lp_uni = lprobust(y, x, eval=eval_points, h=0.15, p=1, kernel="uni", vce="nn")

print("="*70)
print("TEST 2: lprobust with Uniform kernel")
print("="*70)
result_lp_uni.summary()

---
## Test 3: lprobust with Triangular Kernel

In [ ]:
result_lp_tri = lprobust(y, x, eval=eval_points, h=0.15, p=1, kernel="tri", vce="nn")

print("="*70)
print("TEST 3: lprobust with Triangular kernel")
print("="*70)
result_lp_tri.summary()

---
## Test 4: lprobust with Higher Polynomial Order (p=2)

In [ ]:
result_lp_p2 = lprobust(y, x, eval=eval_points, h=0.15, p=2, kernel="epa", vce="nn")

print("="*70)
print("TEST 4: lprobust with p=2")
print("="*70)
result_lp_p2.summary()

---
## Test 5: lprobust Derivative Estimation (deriv=1)

In [ ]:
result_lp_d1 = lprobust(y, x, eval=eval_points, h=0.15, p=2, deriv=1, kernel="epa", vce="nn")

print("="*70)
print("TEST 5: lprobust with deriv=1 (first derivative)")
print("="*70)
result_lp_d1.summary()

---
## Test 6: lpbwselect - MSE-DPI Bandwidth Selection

In [ ]:
bw_mse = lpbwselect(y, x, eval=eval_points, bwselect="mse-dpi", p=1, kernel="epa")

print("="*70)
print("TEST 6: lpbwselect with MSE-DPI")
print("="*70)
bw_mse.summary()

In [ ]:
print("\nRaw Bandwidths (for comparison with R):")
df_bw = pd.DataFrame(bw_mse.bws, columns=['eval', 'h', 'b'])
print(df_bw.to_string(index=False))

---
## Test 7: lpbwselect - IMSE-DPI Bandwidth Selection

In [ ]:
bw_imse = lpbwselect(y, x, bwselect="imse-dpi", p=1, kernel="epa")

print("="*70)
print("TEST 7: lpbwselect with IMSE-DPI")
print("="*70)
bw_imse.summary()

---
## Test 8: lprobust with Automatic Bandwidth Selection

In [ ]:
result_lp_auto = lprobust(y, x, eval=eval_points, p=1, kernel="epa", bwselect="mse-dpi")

print("="*70)
print("TEST 8: lprobust with automatic bandwidth (MSE-DPI)")
print("="*70)
result_lp_auto.summary()

---
## Test 9: kdrobust - Kernel Density Estimation with Fixed Bandwidth

In [ ]:
# Evaluation points for density
eval_kd = np.array([0.1, 0.3, 0.5, 0.7, 0.9])

result_kd = kdrobust(x, eval=eval_kd, h=0.1, kernel="epa")

print("="*70)
print("TEST 9: kdrobust(x, eval=c(0.1,0.3,0.5,0.7,0.9), h=0.1)")
print("="*70)
result_kd.summary()

In [ ]:
print("\nRaw Estimates (for comparison with R):")
df_kd = pd.DataFrame(result_kd.Estimate, columns=['eval', 'h', 'b', 'N', 'f.us', 'f.bc', 'se.us', 'se.rb'])
print(df_kd.to_string(index=False))

---
## Test 10: kdrobust with Gaussian Kernel

In [ ]:
result_kd_gau = kdrobust(x, eval=eval_kd, h=0.1, kernel="gau")

print("="*70)
print("TEST 10: kdrobust with Gaussian kernel")
print("="*70)
result_kd_gau.summary()

---
## Test 11: kdbwselect - Bandwidth Selection for KDE

In [ ]:
bw_kd = kdbwselect(x, eval=eval_kd, bwselect="mse-dpi", kernel="epa")

print("="*70)
print("TEST 11: kdbwselect with MSE-DPI")
print("="*70)
bw_kd.summary()

---
## Test 12: kdrobust with Automatic Bandwidth

In [ ]:
result_kd_auto = kdrobust(x, eval=eval_kd, kernel="epa", bwselect="mse-dpi")

print("="*70)
print("TEST 12: kdrobust with automatic bandwidth (MSE-DPI)")
print("="*70)
result_kd_auto.summary()

---
## Test 13: lprobust with HC Variance Estimators

In [ ]:
# Test with HC0
result_hc0 = lprobust(y, x, eval=eval_points, h=0.15, p=1, kernel="epa", vce="hc0")

print("="*70)
print("TEST 13a: lprobust with vce='hc0'")
print("="*70)
result_hc0.summary()

In [ ]:
# Test with HC1
result_hc1 = lprobust(y, x, eval=eval_points, h=0.15, p=1, kernel="epa", vce="hc1")

print("="*70)
print("TEST 13b: lprobust with vce='hc1'")
print("="*70)
result_hc1.summary()

In [ ]:
# Test with HC2
result_hc2 = lprobust(y, x, eval=eval_points, h=0.15, p=1, kernel="epa", vce="hc2")

print("="*70)
print("TEST 13c: lprobust with vce='hc2'")
print("="*70)
result_hc2.summary()

---
## Summary Table for Easy Comparison

In [ ]:
print("\n" + "="*80)
print("SUMMARY OF KEY RESULTS FOR R COMPARISON")
print("="*80)

print("\n1. lprobust (h=0.15, p=1, epa, vce=nn):")
print(pd.DataFrame(result_lp.Estimate, columns=['eval', 'h', 'b', 'N', 'tau.us', 'tau.bc', 'se.us', 'se.rb']).round(6).to_string(index=False))

print("\n2. lpbwselect (mse-dpi):")
print(pd.DataFrame(bw_mse.bws, columns=['eval', 'h', 'b']).round(6).to_string(index=False))

print("\n3. kdrobust (h=0.1, epa):")
print(pd.DataFrame(result_kd.Estimate, columns=['eval', 'h', 'b', 'N', 'f.us', 'f.bc', 'se.us', 'se.rb']).round(6).to_string(index=False))

print("\n4. kdbwselect (mse-dpi):")
print(pd.DataFrame(bw_kd.bws, columns=['eval', 'h', 'b']).round(6).to_string(index=False))

---
## Visualization (Optional)

In [ ]:
try:
    import matplotlib.pyplot as plt
    
    # Create figure with subplots
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Plot 1: Local polynomial regression
    ax1 = axes[0]
    ax1.scatter(x, y, alpha=0.3, s=10, label='Data')
    
    # Get more evaluation points for smooth curve
    eval_smooth = np.linspace(0.05, 0.95, 50)
    result_smooth = lprobust(y, x, eval=eval_smooth, h=0.15, p=1, kernel="epa")
    
    ax1.plot(result_smooth.Estimate[:, 0], result_smooth.Estimate[:, 4], 'b-', linewidth=2, label='LP Estimate')
    ax1.fill_between(result_smooth.Estimate[:, 0], 
                     result_smooth.Estimate[:, 5] - 1.96*result_smooth.Estimate[:, 7],
                     result_smooth.Estimate[:, 5] + 1.96*result_smooth.Estimate[:, 7],
                     alpha=0.2, color='blue', label='95% CI')
    
    # True function
    x_true = np.linspace(0, 1, 100)
    ax1.plot(x_true, np.sin(2*np.pi*x_true), 'r--', linewidth=1.5, label='True function')
    
    ax1.set_xlabel('x')
    ax1.set_ylabel('y')
    ax1.set_title('Local Polynomial Regression (p=1, h=0.15)')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # Plot 2: Kernel density estimation
    ax2 = axes[1]
    ax2.hist(x, bins=30, density=True, alpha=0.3, label='Histogram')
    
    eval_kd_smooth = np.linspace(0.05, 0.95, 50)
    result_kd_smooth = kdrobust(x, eval=eval_kd_smooth, h=0.1, kernel="epa")
    
    ax2.plot(result_kd_smooth.Estimate[:, 0], result_kd_smooth.Estimate[:, 4], 'b-', linewidth=2, label='KDE Estimate')
    ax2.fill_between(result_kd_smooth.Estimate[:, 0],
                     result_kd_smooth.Estimate[:, 5] - 1.96*result_kd_smooth.Estimate[:, 7],
                     result_kd_smooth.Estimate[:, 5] + 1.96*result_kd_smooth.Estimate[:, 7],
                     alpha=0.2, color='blue', label='95% CI')
    
    # True density (uniform)
    ax2.axhline(y=1, color='r', linestyle='--', linewidth=1.5, label='True density')
    
    ax2.set_xlabel('x')
    ax2.set_ylabel('Density')
    ax2.set_title('Kernel Density Estimation (h=0.1)')
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig('nprobust_python_results.png', dpi=150, bbox_inches='tight')
    plt.show()
    
    print("\nPlot saved to nprobust_python_results.png")
    
except ImportError:
    print("matplotlib not installed. Skipping visualization.")

---
## Save Results for Comparison

In [ ]:
# Save Python results to CSV files for detailed comparison
pd.DataFrame(result_lp.Estimate, 
             columns=['eval', 'h', 'b', 'N', 'tau.us', 'tau.bc', 'se.us', 'se.rb']
            ).to_csv('python_lprobust_results.csv', index=False)

pd.DataFrame(bw_mse.bws, 
             columns=['eval', 'h', 'b']
            ).to_csv('python_lpbwselect_results.csv', index=False)

pd.DataFrame(result_kd.Estimate, 
             columns=['eval', 'h', 'b', 'N', 'f.us', 'f.bc', 'se.us', 'se.rb']
            ).to_csv('python_kdrobust_results.csv', index=False)

pd.DataFrame(bw_kd.bws, 
             columns=['eval', 'h', 'b']
            ).to_csv('python_kdbwselect_results.csv', index=False)

print("Results saved to:")
print("  - python_lprobust_results.csv")
print("  - python_lpbwselect_results.csv")
print("  - python_kdrobust_results.csv")
print("  - python_kdbwselect_results.csv")

---
## Instructions for R Comparison

Now run the companion R script `comparison_test.R` which performs the same tests.

```r
source("comparison_test.R")
```

The R script will produce similar output that you can compare directly with the Python results above.